In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker
import seaborn as sns
%load_ext autoreload
%autoreload 2

from settings import *
from glove import *
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from textblob import TextBlob
from tweets_ops import *
from url_helpers import analyze_url
from matching import *
from textstat.textstat import textstat
import urllib, bs4
pd.options.mode.chained_assignment = None

In [2]:
#DailyMail -> Bad
#NYTimes -> Neutral for acsh Top for climatefeedback
#the Atlantic -> Top for acsh Neutral for climatefeedback

In [3]:
df = pd.read_csv(cache_dir+'article_details_v5.tsv', sep='\t')

In [4]:
df[df.apply(lambda x: analyze_url(x['url'])[0]=='theatlantic.com', axis=1)].to_csv(cache_dir+'theatlantic.tsv', sep='\t', index=None)

aggregate_tweet_details(cache_dir+'diffusion_graph/pruned_graph_v3.tsv', cache_dir+'tweet_details_v3.tsv', cache_dir+'theatlantic.tsv', cache_dir+'theatlantic.tsv')


In [5]:
df = pd.read_csv(cache_dir+'theatlantic.tsv', sep='\t').fillna(0)

In [6]:
case = 'theatlantic'
for s in ['full', 'paragraph', 'sentence']:
    train = pd.read_csv(cache_dir+'similarity_model/train_pairs_v2_'+s+'.tsv', sep='\t').merge(df[['url']], left_on='article', right_on='url').drop('url', axis=1)
    train = train[train.related==True]
    test = pd.read_csv(cache_dir+'similarity_model/test_pairs_v2_'+s+'.tsv', sep='\t').merge(df[['url']], left_on='article', right_on='url').drop('url', axis=1)
    pd.concat([train, test]).to_csv(cache_dir+'similarity_model/'+case+'/'+s+'.tsv', sep='\t', index=None)
    
test_similarity_model(cache_dir+'similarity_model/'+case+'/', cache_dir+'similarity_model/rf_model.sav', cache_dir+'similarity_model/'+case+'/results.tsv')
sim = pd.read_csv(cache_dir+'similarity_model/'+case+'/results.tsv', sep='\t').drop('paper', axis=1).groupby('article').max().reset_index()
df = df.merge(sim, left_on='url', right_on='article').drop('article', axis=1)

/opt/anaconda3/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.19.1 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.19.1 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.19.1 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.19.1 when using version 0.20.0. This might lead to breakin

In [7]:
#df[['url', 'title', 'authors', 'quote_indicators', 'likes', 'replies',
#        'replies_count', 'replies_mean_polarity', 'replies_mean_subjectivity',
#        'retweets', 'tweets_mean_polarity', 'tweets_mean_subjectivity',
#        'tweets_time_delta', 'users_countries', 'users_median_followers',
#        'users_median_friends', 'users_median_tweets', 'related']]



In [8]:
df['readability'] = df['full_text'].apply(lambda x: textstat.flesch_reading_ease(x))
df['title_subjectivity'] = df['title'].apply(lambda x: TextBlob(x).subjectivity)
df['title_polarity'] = df['title'].apply(lambda x: TextBlob(x).polarity)

In [9]:
df = pd.concat([df.drop(['quote_indicators'], axis=1), df['quote_indicators'].apply(lambda x: pd.Series(eval(x)))], axis=1)


In [10]:
df['has_author'] = df['authors'].isnull() | (df['authors'] == '')

In [11]:
df.columns

Index(['url', 'title', 'authors', 'keywords', 'publish_date', 'full_text',
       'entities', 'topics', 'quotes', 'likes', 'replies_count',
       'replies_mean_polarity', 'replies_mean_subjectivity', 'retweets',
       'stance', 'tweets_mean_polarity', 'tweets_mean_subjectivity',
       'tweets_time_delta', 'users_countries', 'users_median_followers',
       'users_median_friends', 'users_median_tweets', 'related', 'readability',
       'title_subjectivity', 'title_polarity', 'count_all_quotes',
       'count_PER_quotes', 'count_ORG_quotes', 'count_unnamed_quotes',
       'has_author'],
      dtype='object')

In [12]:
G = read_graph(cache_dir+'diffusion_graph/pruned_graph_v3.tsv')
pr = nx.pagerank(G.reverse())

In [13]:
df['pageRank'] = df['url'].apply(lambda x: pr[x])

In [14]:
df['word_count'] = df['full_text'].apply(lambda x: len(re.findall(r'\w+', x)))

In [15]:
df['alexa_rank']=df['url'].apply(lambda x: bs4.BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url="+str(x)).read(), "xml").find("REACH")['RANK'])


In [16]:
df.url = df.url.apply(lambda x: analyze_url(x)[0])

In [17]:
df[['url', 'likes', 'replies_count',
    'replies_mean_polarity', 'replies_mean_subjectivity', 'retweets',
    'stance', 'tweets_mean_polarity', 'tweets_mean_subjectivity',
    'tweets_time_delta', 'users_countries', 'users_median_followers',
    'users_median_friends', 'users_median_tweets', 'related', 'readability',
    'title_subjectivity', 'title_polarity', 'count_all_quotes',
    'count_PER_quotes', 'count_ORG_quotes', 'count_unnamed_quotes',
    'has_author', 'pageRank', 'word_count', 'alexa_rank']].to_csv(cache_dir+'theatlantic.tsv', sep='\t', index=None)

In [25]:
df = pd.concat([pd.read_csv(cache_dir+'theatlantic.tsv', sep='\t'), pd.read_csv(cache_dir+'nytimes.tsv', sep='\t'), pd.read_csv(cache_dir+'dailymail.tsv', sep='\t')])
y = df[['url']].values
X = np.array(df.drop('url', axis=1).values, dtype=np.float32)

In [39]:
n_est = 100
m_dep = 10
X = (X - X.mean(axis=0)) / (X.std(axis=0)+1e-9)
classifier = RandomForestClassifier(n_estimators=n_est, max_depth=m_dep, n_jobs=-1)
classifier.fit(X, y)
result = classifier.feature_importances_

/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


In [41]:
from sklearn.feature_selection import f_classif
_, result = f_classif(X,y)

/opt/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: UserWarning: Features [21 24] are constant.
  UserWarning)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/opt/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [42]:
d = {c:r for c, r in zip(df.columns[1:], result)}
sorted(d.items(), key=lambda kv: kv[1])

[('alexa_rank', 0.0),
 ('count_ORG_quotes', 3.5488284e-07),
 ('word_count', 0.0060870694),
 ('count_all_quotes', 0.011927988),
 ('title_polarity', 0.020459978),
 ('replies_count', 0.12580314),
 ('count_unnamed_quotes', 0.12993713),
 ('tweets_mean_polarity', 0.13073456),
 ('replies_mean_subjectivity', 0.25727576),
 ('tweets_mean_subjectivity', 0.33654013),
 ('users_median_followers', 0.3874909),
 ('tweets_time_delta', 0.5226302),
 ('users_countries', 0.57485324),
 ('replies_mean_polarity', 0.58907974),
 ('pageRank', 0.6001142),
 ('retweets', 0.72130746),
 ('users_median_friends', 0.7357557),
 ('stance', 0.7742686),
 ('title_subjectivity', 0.80346787),
 ('related', 0.8035081),
 ('readability', 0.85317963),
 ('likes', 0.8921011),
 ('count_PER_quotes', 0.95182353),
 ('users_median_tweets', 0.99825907),
 ('has_author', nan)]